# Output Parsers

**Output Parsers** in LangChain help convert raw LLM responses into structured formats such as JSON, CSV, Pydantic models, and more.  
They ensure consistency, validation, and ease of use in applications. 

- StrOutputParser
- JSONOutputParser
- StructuredOutputParser
- PydanticOutputParser

In [19]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from dotenv import load_dotenv
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="MiniMaxAI/MiniMax-M2.1",
    task="text-generation",
    max_new_tokens=1000,
)

model = ChatHuggingFace(llm=llm)

## StrOutputParser

The **StrOutputParser** is the simplest output parser in LangChain.  
It is used to parse the output of a Language Model (LLM) and return it as a plain string.


#### Without `StrOutputParser`

In [22]:
from langchain_core.prompts import PromptTemplate

# First prompt, Detailed Report
promptDetailed = PromptTemplate(
    template='Write detailed report on {topic} in about 700 words.',
    input_variables=['topic']
)

# Second prompt, summary of report
promptSummary = PromptTemplate(
    template='Write seven line summary on the following text. \n {text}',
    input_variables=['text']
)

detailed = promptDetailed.invoke({'topic':"Black hole"})

detailedRes = model.invoke(detailed)
print('Report Generated') if detailedRes.content else 'Failed to generate report'

summary = promptSummary.invoke({'text':detailedRes.content})

summaryRes = model.invoke(summary)
print('Summary Generated') if summaryRes.content else 'Failed to generate summary'
print(summaryRes.content) if summaryRes.content else ''

Report Generated
Summary Generated
# Seven-Line Summary: Black Holes

Black holes are cosmic regions where gravity is so extreme that nothing—not even light—can escape once past the event horizon, created when massive stars collapse or gas clouds compress. These enigmatic objects feature a singularity of infinite density at their center, surrounded by the event horizon, the boundary beyond which escape becomes mathematically impossible. Astronomers classify black holes into three main types: stellar-mass black holes from collapsed massive stars, supermassive black holes containing millions to billions of solar masses at galactic centers, and intermediate-mass black holes filling the middle range. Since black holes emit no light themselves, detection relies on observing their effects on surrounding matter, such as accretion disks emitting X-rays and gravitational influences on nearby stars. The 2015 detection of gravitational waves by LIGO and the 2019 Event Horizon Telescope image of M

#### With `StrOutputParser`

In [25]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# First prompt, Detailed Report
promptDetailed = PromptTemplate(
    template='Write detailed report on {topic} in about 700 words.',
    input_variables=['topic']
)

# Second prompt, summary of report
promptSummary = PromptTemplate(
    template='Write seven bullet point summary on the following text. \n {text}',
    input_variables=['text']
)

parser = StrOutputParser()

chain = promptDetailed | model | parser | promptSummary | model | parser

result = chain.invoke({'topic':"Black hole"})

print(result)

# Seven Bullet Point Summary: Black Holes

- Black holes are regions of spacetime where matter has been compressed to extraordinary densities, creating gravitational fields so intense that nothing—not even light—can escape once it crosses the event horizon, making them among the most enigmatic and powerful phenomena in the universe.

- The most common formation mechanism occurs when massive stars exhaust their nuclear fuel and can no longer sustain outward pressure, causing the core to implode; if the remaining mass exceeds approximately three solar masses, the collapse continues indefinitely into a singularity surrounded by the event horizon.

- Black holes come in three primary varieties: stellar-mass black holes formed from collapsed stars, supermassive black holes containing millions or billions of solar masses that reside at galactic centers (including Sagittarius A* in our own Milky Way), and intermediate-mass black holes whose formation pathways remain debated among astronomers.

## JSONOutputParser

The **JSONOutputParser** is an output parser in LangChain used to convert the output of a Language Model (LLM) into a valid JSON object.  
It helps ensure the response follows a structured JSON format, making it easier to validate and process programmatically.


In [26]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

template = PromptTemplate(
    template='Give me name, age, city of a fictional persion \n {format_instruction}',
    input_variables=[],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

prompt = template.format()

result = model.invoke(prompt)

In [27]:
result

AIMessage(content='```json\n{\n  "name": "Emma Thompson",\n  "age": 28,\n  "city": "Portland"\n}\n```', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 54, 'total_tokens': 122}, 'model_name': 'MiniMaxAI/MiniMax-M2.1', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b9245-c8a8-77c3-ac05-e420faaaa97a-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 54, 'output_tokens': 68, 'total_tokens': 122})

In [29]:
parser.parse(result.content)

{'name': 'Emma Thompson', 'age': 28, 'city': 'Portland'}

In [30]:
chain = template | model | parser

result = chain.invoke({})
result

{'name': 'Liam Harper', 'age': 32, 'city': 'Portland'}

## StructuredOutputParser

**StructuredOutputParser** is an output parser in LangChain that helps extract structured JSON data from LLM responses based on predefined field schemas.

It works by defining a list of fields (`ResponseSchema`) that the model should return, ensuring the output follows a structured format.


## PydanticOutputParser

**PydanticOutputParser** is a structured output parser in LangChain that uses **Pydantic models** to enforce schema validation when processing LLM responses.

### Why Use PydanticOutputParser?

- ✅ **Strict Schema Enforcement** – Ensures that LLM responses follow a well-defined structure.
- ✅ **Type Safety** – Automatically converts LLM outputs into Python objects.
- ✅ **Easy Validation** – Uses Pydantic’s built-in validation to catch incorrect or missing data.
- ✅ **Seamless Integration** – Works well with other LangChain components.


In [57]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class Person(BaseModel):
    name: str = Field(description="Name of the person")
    age: int = Field(gt=18, description='Age of the person')
    city: str = Field(description='Name of the city the person belong to')

parser = PydanticOutputParser(pydantic_object=Person)

template = PromptTemplate(
    template='Generate the name, age, city of a fictional person live in {country} \n {format_instruction}',
    input_variables=['country'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

prompt = template.invoke({'country':'India'})

result = model.invoke(prompt)

print(parser.parse(result.content))

name='Priya Sharma' age=28 city='Bangalore'


In [58]:
chain = template | model | parser
res = chain.invoke({'country':'India'})
print(res)

name='Rahul Sharma' age=28 city='Mumbai'
